elapiezo:
example runs fine, change the geometry it stops

# Effective mass 

- grid of k points in 3D
    - input:
        - center point
        - number of points
        - max distance (spacing)
    - output:
        - k point coordinates (frac)

- energy of points on the grid
    - input:
        - fractional coordinates
        - band number
        - CRYSTALpytools -> shrinking factor
        - CRYSTALpytools -> band structure input file 1 per k-point
    - output:
        - a lot of .BANDS files
- script to extract the energy from .BANDS files
- calculate 2nd derivatives using finite differences


In [30]:
import numpy as np
from CRYSTALpytools.crystal_io import *
from CRYSTALpytools.convert import *

In [174]:
import numpy as np

def generate_3d_integer_grid(max_values):
    # Generate 1D arrays of integer points for each dimension
    x = np.arange(-max_values[0], max_values[0] + 1, dtype=int)
    y = np.arange(-max_values[1], max_values[1] + 1, dtype=int)
    z = np.arange(-max_values[2], max_values[2] + 1, dtype=int)

    # Create a mesh grid
    x_grid, y_grid, z_grid = np.meshgrid(x, y, z, indexing='ij')

    # Reshape the grids to 1D arrays and stack them
    grid_points = np.column_stack((x_grid.ravel(), y_grid.ravel(), z_grid.ravel()))

    return grid_points

# Example usage
max_value = 2
max_values = [max_value,max_value,max_value]


grid_points = generate_3d_integer_grid(max_values)

print(np.lcm.reduce(np.unique(grid_points)[1:]))
print("Generated 3D integer grid points:")
print(np.max(grid_points/6))

0
Generated 3D integer grid points:
0.3333333333333333


In [175]:
grid_points

array([[-2, -2, -2],
       [-2, -2, -1],
       [-2, -2,  0],
       [-2, -2,  1],
       [-2, -2,  2],
       [-2, -1, -2],
       [-2, -1, -1],
       [-2, -1,  0],
       [-2, -1,  1],
       [-2, -1,  2],
       [-2,  0, -2],
       [-2,  0, -1],
       [-2,  0,  0],
       [-2,  0,  1],
       [-2,  0,  2],
       [-2,  1, -2],
       [-2,  1, -1],
       [-2,  1,  0],
       [-2,  1,  1],
       [-2,  1,  2],
       [-2,  2, -2],
       [-2,  2, -1],
       [-2,  2,  0],
       [-2,  2,  1],
       [-2,  2,  2],
       [-1, -2, -2],
       [-1, -2, -1],
       [-1, -2,  0],
       [-1, -2,  1],
       [-1, -2,  2],
       [-1, -1, -2],
       [-1, -1, -1],
       [-1, -1,  0],
       [-1, -1,  1],
       [-1, -1,  2],
       [-1,  0, -2],
       [-1,  0, -1],
       [-1,  0,  0],
       [-1,  0,  1],
       [-1,  0,  2],
       [-1,  1, -2],
       [-1,  1, -1],
       [-1,  1,  0],
       [-1,  1,  1],
       [-1,  1,  2],
       [-1,  2, -2],
       [-1,  2, -1],
       [-1,  

In [178]:
k_unique = [0.5,0.25]
k_unique = np.array(np.around(k_unique, 4)*10000, dtype=int)
if len(k_unique) > 2:
    gcd = np.gcd.reduce(k_unique)
else:
    gcd = np.gcd(k_unique[0], k_unique[1])
k_path = np.array((k_path/gcd)*10000, dtype=int)
shrink = int(10000/gcd)

/var/folders/f2/8kc7y9697m59bwltxjd42y300000gn/T/ipykernel_10743/496468699.py:7: RuntimeWarning: invalid value encountered in cast
  k_path = np.array((k_path/gcd)*10000, dtype=int)


In [134]:
center = np.array([0, 0, 0])
num_points = 2
np.linspace(center[0] +num_points, center[0] +num_points, num_points)

array([2., 2.])

In [126]:

def generate_3d_grid(center, num_points, max_distance):
    
    if num_points%2 == 0:
        num_points += 1
    # Generate a mesh grid of coordinates
    x = np.linspace(center[0] - max_distance, center[0] + max_distance, num_points)
    y = np.linspace(center[1] - max_distance, center[1] + max_distance, num_points)
    z = np.linspace(center[2] - max_distance, center[2] + max_distance, num_points)

    # Create a mesh grid
    x_grid, y_grid, z_grid = np.meshgrid(x, y, z)

    # Reshape the grids to 1D arrays
    x_points = x_grid.ravel()
    y_points = y_grid.ravel()
    z_points = z_grid.ravel()

    # Filter points based on distance from the center
    distances = np.sqrt((x_points - center[0])**2 + (y_points - center[1])**2 + (z_points - center[2])**2)
    valid_points = distances <= max_distance

    # Return the filtered coordinates
    return np.column_stack((x_points[valid_points], y_points[valid_points], z_points[valid_points]))

# Example usage
center_coordinate = np.array([0.0, 0.0, 0.0])
num_points = 4
max_distance = 1

grid_points = generate_3d_grid(center_coordinate, num_points, max_distance)
print("Generated 3D grid points:")
print(grid_points)

Generated 3D grid points:
[[ 0.  -1.   0. ]
 [-0.5 -0.5 -0.5]
 [-0.5 -0.5  0. ]
 [-0.5 -0.5  0.5]
 [ 0.  -0.5 -0.5]
 [ 0.  -0.5  0. ]
 [ 0.  -0.5  0.5]
 [ 0.5 -0.5 -0.5]
 [ 0.5 -0.5  0. ]
 [ 0.5 -0.5  0.5]
 [-1.   0.   0. ]
 [-0.5  0.  -0.5]
 [-0.5  0.   0. ]
 [-0.5  0.   0.5]
 [ 0.   0.  -1. ]
 [ 0.   0.  -0.5]
 [ 0.   0.   0. ]
 [ 0.   0.   0.5]
 [ 0.   0.   1. ]
 [ 0.5  0.  -0.5]
 [ 0.5  0.   0. ]
 [ 0.5  0.   0.5]
 [ 1.   0.   0. ]
 [-0.5  0.5 -0.5]
 [-0.5  0.5  0. ]
 [-0.5  0.5  0.5]
 [ 0.   0.5 -0.5]
 [ 0.   0.5  0. ]
 [ 0.   0.5  0.5]
 [ 0.5  0.5 -0.5]
 [ 0.5  0.5  0. ]
 [ 0.5  0.5  0.5]
 [ 0.   1.   0. ]]


In [130]:
np.unique(grid_points)
np.gcd.reduce(np.unique(grid_points))

UFuncTypeError: ufunc 'gcd' did not contain a loop with signature matching types (None, <class 'numpy.dtype[float64]'>) -> None

In [119]:
k_unique = np.unique(grid_points)
k_unique = np.array(np.around(k_unique, 4)*1000, dtype=int)
if len(k_unique) > 2:
    gcd = np.gcd.reduce(k_unique)
else:
    gcd = np.gcd(k_unique[0], k_unique[1])
k_path = np.array((k_path/gcd)*10000, dtype=int)
shrink = int(10000/gcd)

/var/folders/f2/8kc7y9697m59bwltxjd42y300000gn/T/ipykernel_10743/143467000.py:7: RuntimeWarning: invalid value encountered in cast
  k_path = np.array((k_path/gcd)*10000, dtype=int)


In [120]:
k_unique = np.unique(grid_points)

# Find the shrinking factor
k_unique = np.array(np.around(k_unique, 4)*10000, dtype=int)
if len(k_unique) > 2:
    gcd = np.gcd.reduce(k_unique)
else:
    gcd = np.gcd(k_unique[0], k_unique[1])
k_path = np.array((k_path/gcd)*10000, dtype=int)
shrink = int(10000/gcd)

/var/folders/f2/8kc7y9697m59bwltxjd42y300000gn/T/ipykernel_10743/4053649238.py:9: RuntimeWarning: invalid value encountered in cast
  k_path = np.array((k_path/gcd)*10000, dtype=int)


In [110]:
k_unique

array([49, 49, 50, 50, 50])

In [105]:
k_path

array([        0, 100000000])

In [102]:
band_input = Properties_input().make_bands_block(grid_points.tolist(),1,23,23)
band_input.property_block

['BAND\n',
 'BAND STRUCTURE CALCULATION\n',
 '98 10000 1 23 23 1 0\n',
 '4993 4993 4996  4993 4993 5000\n',
 '4993 4993 5000  4993 4993 5003\n',
 '4993 4993 5003  4996 4993 4993\n',
 '4996 4993 4993  4996 4993 4996\n',
 '4996 4993 4996  4996 4993 5000\n',
 '4996 4993 5000  4996 4993 5003\n',
 '4996 4993 5003  5000 4993 4993\n',
 '5000 4993 4993  5000 4993 4996\n',
 '5000 4993 4996  5000 4993 5000\n',
 '5000 4993 5000  5000 4993 5003\n',
 '5000 4993 5003  5000 4993 5006\n',
 '5000 4993 5006  5003 4993 4993\n',
 '5003 4993 4993  5003 4993 4996\n',
 '5003 4993 4996  5003 4993 5000\n',
 '5003 4993 5000  5003 4993 5003\n',
 '5003 4993 5003  5006 4993 5000\n',
 '5006 4993 5000  4993 4996 4993\n',
 '4993 4996 4993  4993 4996 4996\n',
 '4993 4996 4996  4993 4996 5000\n',
 '4993 4996 5000  4993 4996 5003\n',
 '4993 4996 5003  4996 4996 4993\n',
 '4996 4996 4993  4996 4996 4996\n',
 '4996 4996 4996  4996 4996 5000\n',
 '4996 4996 5000  4996 4996 5003\n',
 '4996 4996 5003  4996 4996 5006\n',
 '49

In [65]:
1/7

0.14285714285714285

In [ ]:
make_bands_block(k_path, n_kpoints, first_band, last_band, print_eig=0, print_option=1, title='BAND STRUCTURE CALCULATION')

In [197]:
10**4

10000

In [235]:
n_zeros = 5
k_unique = [0,0.082751,0.5]
#k_path = k_unique
# Find the shrinking factor
k_unique = np.array(np.around(k_unique, n_zeros)*10**n_zeros, dtype=int)
print(k_unique)
if len(k_unique) > 2:
    gcd = np.gcd.reduce(k_unique)
    print(gcd)
else:
    gcd = np.gcd(k_unique[0], k_unique[1])
    print(gcd)
k_path = np.array((k_unique/gcd)*10**n_zeros, dtype=int)
shrink = int(10**n_zeros/gcd)
print(shrink)

[    0  8275 50000]
25
4000


In [234]:
k_path/shrink

array([    0.,  8275., 50000.])

In [93]:
from fractions import Fraction
import numpy as np

def find_gcd_and_scaled_numbers(fractional_numbers, decimal_places=4):
    # Convert fractional numbers to Fraction objects
    fractions = [Fraction(num).limit_denominator(10**decimal_places) for num in fractional_numbers]

    # Extract numerators
    numerators = [frac.numerator for frac in fractions]

    # Find the gcd of numerators
    if len(numerators) > 2:
        gcd = np.gcd.reduce(numerators)
    else:
        gcd = np.gcd(numerators[0], numerators[1])

    # Calculate the scaling factor
    scaling_factor = int(10**decimal_places) // gcd

    # Scale the numbers to integers
    scaled_numbers = [int(num * scaling_factor) for num in fractional_numbers]

    return gcd, scaled_numbers

# Example usage
fractional_numbers = [0.1, 0.2, 0.3]
decimal_places = 6  # Set the desired number of decimal places

gcd, scaled_numbers = find_gcd_and_scaled_numbers(fractional_numbers, decimal_places)

print("Fractional Numbers:", fractional_numbers)
print("GCD:", gcd)
print("Scaled Numbers:", scaled_numbers)


Fractional Numbers: [0.1, 0.2, 0.3]
GCD: 1
Scaled Numbers: [100000, 200000, 300000]
